# Answer to question 1:

Before answering, it is noted that PostGres Database has PosGIS extension, made for spatial indexing.
https://postgis.net/workshops/postgis-intro/indexing.html


My approach to solving the problem is to store the information in a dictionary, where the keys are a user, and the values is another dictionary, where the keys are a tuple of longitude and latitude.

for e.g

{ user1: {}, user2: {}, user3: {} }

The values in turn are another dictionary, who's keys are tuples of longitude and latitude.

For e.g.

{ (lat1,long1): True, (lat2,long1): True }


The reason why these values are stored in dictionary gives us efficient access to the key of O(1).

The caveat to this problem is that latitude and longitude reading are inaccurate, giving us different values.
It becomes unneccessary to store all the longitude and latitude points since one location may contain many longitude and latitude points. To solve this, when we insert a new location into our dictionary, we first see if any other location points are within M=500 meters of the proposed point.If it is then we don't store the point. The distance formula used is the Haversine formula implemented in our distance function. 


Doing this will on-average save memory of our dictionary. However, this memory saving implementation comes at compute disadvantage. On the worst case it cases O(n^2) where to create this dictionary where n is the number (latitude,longitude) points. Also on the worst case our memory could still be O(n) where every point is 500m apart from each other.However, on the average case it stores a lot less location readings. For e.g. On the first csv. We went from 700 rows to less than 300 rows after we took into account distance between points.


# Lookup algorithm:
The algorithm for lookup goes as the following. First check to see if there is a key with location being inquired on. If not, then we loop through the keys and compute a distance measure with every existing key in the dictionary. If the distance measure is less than M=500m, then we would return true. This is because since longitude and latitude readings maybe be innacurrate, we allow a "grace" distance to make up for the error. Otherwise we would return false. This code is implemented in the lookup function.


# Analysis of Lookup:

Best Case Scenario:O(1)
 Imagine a user has visited exactly a longitude and latitude point. This would take O(2). O(1)(lookup user) + O(1) (look up longitude and latitude).
 
 
# Worst Case:O(n)
   This the case where the longitude and latitude are not exactly in the dictionary or the case where no point is 500m within our proposed location. Since there are up to n points in our dictionary, we would compare with n points thus O(n).
   
  
# Average Case:
    Remember that our dictionary doesn't necessarily store all the points. Thus the n points in our dictionary maybe be much less than the points in our csv files.


How to use the Class. 

Our data_table is enscapulated within our python Data_Lookup Class. For your ease I've included the class in this jupyter notebook. Once you initialize the class, everything is set up for you to use the lookup function. The lookup function accepts a tuple of (latitude,longitude) and the user for who you want to search for.The user corresponds to the name of the csv file provided. I.e person1.csv -> person1

E.G.

dataTable = dataLookUp()

dataTable.lookup((51.4513,4.88882),"person1") -> True or False


I've also included some unit tests below.



In [10]:
from math import radians, cos, sin, asin, sqrt
import matplotlib.pyplot as plot
import datetime
from datetime import timedelta
import pandas as pd


class dataLookUp:

    def __init__(self):
        self.data_table = {}
        self.list_of_csvs = ["person1.csv", "person2.csv", "person3.csv"]
        self.fill_data_table()

    def distance(self,lat1, lat2, lon1, lon2):
        #Haversine formula (https://en.wikipedia.org/wiki/Haversine_formula)


        # The math module contains a function named
        # radians which converts from degrees to radians.
        lon1 = radians(lon1)
        lon2 = radians(lon2)
        lat1 = radians(lat1)
        lat2 = radians(lat2)

        # Haversine formula
        dlon = lon2 - lon1
        dlat = lat2 - lat1
        a = sin(dlat / 2) ** 2 + cos(lat1) * cos(lat2) * sin(dlon / 2) ** 2

        c = 2 * asin(sqrt(a))

        # Radius of earth in kilometers. Use 3956 for miles
        r = 6371

        # calculate the result
        return (c * r)



    def insert_locations(self, csv):
        user = csv[:-4]
        df = pd.read_csv(csv, sep=';')
        self.data_table[user] = {}
        for index, row in df.iterrows():
            unique = True
            latitude = row['latitude']
            longitude = row['longitude']
            for key in self.data_table[user]:
                existing_lat, existing_lon = key[0], key[1]
                dist = self.distance(existing_lat, latitude , existing_lon, longitude)
                if (dist < 1.0):
                    unique = False
                    break
            if (unique):
                self.data_table[user][(latitude, longitude)] = True

    def fill_data_table(self):
        for file in self.list_of_csvs:
            self.insert_locations(file)

    def lookup(self,location,user):
        if location in self.data_table[user]:
            return True
        proposed_lat, proposed_lon = location[0], location[1]
        for key in self.data_table[user]:
            existing_lat, existing_lon = key[0], key[1]
            dist = self.distance(existing_lat, proposed_lat, existing_lon, proposed_lon)
            if (dist < 0.5):
                return True
        return False


In [35]:
import unittest
class TestStringMethods(unittest.TestCase):

        

    def test_exact_location(self):
        table = dataLookUp()
        location_exists = table.lookup((-50.3359, -72.2498), "person1")
        self.assertTrue(location_exists)

    def testCloseLocation(self):
        #tests whether a location that is very close to the existing location exists
        #(-50.339264, -72.263306) exists
        #so this should also exist (-50.339264, -72.263302)
        table = dataLookUp()
        location_exists = table.lookup((-50.339264, -72.263302), "person1")
        self.assertTrue(location_exists)
        
    def test_not_exists(self):
        table = dataLookUp()
        location_exists = (table.lookup((-39.326958000000005, -72.89073), "person1"))
        self.assertFalse(location_exists)
        

     #-39.326958000000005, -72.89073
        
 #-39.326958000000005, -72.89073

if __name__ == '__main__':
    unittest.main(argv=['first-arg-is-ignored'], exit=False)

...
----------------------------------------------------------------------
Ran 3 tests in 0.685s

OK
